In [55]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import pickle
import pandas as pd


####################################################### Metodi da implementare #######################################################

def getName():
    return "Andrea Accornero (1097521), Alessandro Tocco (1097880)"


def preprocess(df, clfName):
    
    if(clfName == 'LR'):
        print('ENTRO IN STANDARD')
        # Features
        X = df.drop("Year", axis=1)
        y = df["Year"]
        scaler = pickle.load(open("standardScaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([X, y], axis = 1)
        return dfNew  
    
    if((clfName == 'RF') or (clfName == 'KNR')):
        print('ENTRO IN MINMAX')
        X = df.drop("Year", axis=1)
        y = df["Year"]
        scaler = pickle.load(open("minMaxScaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([X, y], axis = 1)
        return dfNew  



def load(clfName):
    if (clfName == "LR"):
        print('ENTRO IN LR' )
        clf = pickle.load(open("regression.save", 'rb'))
        return clf
    elif (clfName == "RF"):
        clf = pickle.load(open("randomForestReg.save", 'rb'))
        return clf
    elif (clfName == "KNR"):
        clf = pickle.load(open("kNeighborReg.save", 'rb'))
        return clf
    elif (clfName == "SVR"):
        clf = pickle.load(open("supportVectorReg.save", 'rb'))
        return clf
    elif (clfName == "FF"):
        clf = pickle.load(open("feedForwardNN.save", 'rb'))
        return clf
    elif (clfName == "TB"):
        clf = pickle.load(open("tabNet.save", 'rb'))
        return clf
    elif (clfName == "TF"):
        clf = pickle.load(open("tabTransf.save", 'rb'))
        return clf
    else:
        return None


def predict(df, clfName, clf):
    X = df.drop("Year", axis=1)
    y = df["Year"]
    #print(X.info())
    ypred = clf.predict(X)
    mse = mean_squared_error(y_true=ypred, y_pred=y)
    mae = mean_absolute_error(ypred, y_pred=y)
    mape = mean_absolute_percentage_error(y_true=ypred, y_pred=y)
    r2 = r2_score(ypred, y_pred=y)
    perf = {"mse": mse, "mae": mae, "mape": mape, "r2square": r2}
    return perf
    

#####################################################################################################################################


import pandas as pd

FILENAME = "train.csv"
CLF_NAME_LIST = [ "LR", "KNR"]
df = pd.read_csv(FILENAME)

#Esecuzione degli algoritmi
for modelName in CLF_NAME_LIST:
    dfProcessed = preprocess(df, modelName)
    clf = load(modelName)
    perf = predict(dfProcessed, modelName, clf)
    print("RESULT team: "+str(getName())+" algoName: "+ modelName + " perf: "+ str(perf))



ENTRO IN STANDARD
ENTRO IN LR
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: LR perf: {'mse': 84.44611870757275, 'mae': 6.627242329639595, 'mape': 0.0033192674033565984, 'r2square': -2.262703628831686}
ENTRO IN KNR
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: KNR perf: {'mse': 71.39989981376897, 'mae': 6.0878026622926, 'mape': 0.003051060709539126, 'r2square': -0.9515155053208879}
